In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))

def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

import numpy as np
import ipyvolume as ipv
import ipywidgets as ipw
import scipy
from scipy.spatial.transform import Rotation as rot
from scipy.spatial.distance import cdist
import math
import matplotlib
import matplotlib.pyplot as plt
import random
from random import uniform

def cart2sph(v):
    x,y,z = v
    XsqPlusYsq = x**2 + y**2
    r = np.sqrt(XsqPlusYsq + z**2)                  # r
    elev = math.atan2(z,np.sqrt(XsqPlusYsq))        # theta
    az = math.atan2(y,x)                            # phi
    return np.array([r, elev, az])

def sph2cart(v):
    r, elev, az = v
    x = r * np.sin(elev) * np.cos(az)
    y = r * np.sin(elev) * np.sin(az)
    z = r * np.cos(elev)
    return np.array([x,y,z])

def angle(v1, v2):
    """ Returns the angle in degrees between vectors 'v1' and 'v2'"""
    v1_u = v1/np.linalg.norm(v1)
    v2_u = v2/np.linalg.norm(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))*180/np.pi

def random_point_on_circle(r):
    angle = np.random.uniform(0.,2.*np.pi)
    x = np.cos(angle)*r 
    y = np.sin(angle)*r
    return np.array([x,y])

def random_point_on_lattice(r):
    choices = np.array([[0,1],[0,-1],[1,0],[-1,0]])
    idx = np.random.choice(len(choices),1)
    return choices[idx]*r


def random_point_on_shpere(R):
    """ Returns points on a sphere with radius R, uniformly distributed"""
    radius = R*(np.random.uniform(0.,1.))**(1./3.)
    theta = np.arccos(np.random.uniform(-1.,1.))
    phi = np.random.uniform(0.,2.*np.pi)
    v = sph2cart([radius,theta,phi])
    return v

def regular_points_on_sphere(n,r):
    coords = []
    alpha = 4.0*np.pi*r*r/n
    d = np.sqrt(alpha)
    m_nu = int(np.round(np.pi/d))
    d_nu = np.pi/m_nu
    d_phi = alpha/d_nu
    count = 0
    for m in range (0,m_nu):
        nu = np.pi*(m+0.5)/m_nu
        m_phi = int(np.round(2*np.pi*np.sin(nu)/d_phi))
        for n in range (0,m_phi):
            phi = 2*np.pi*n/m_phi
            point = sph2cart([r,nu,phi])
            coords.append(point)
    return np.asarray(coords)

def random_point_on_cone(R,theta,prev):
    """ Returns random vector with length R and angle theta between previos one, uniformly distributed"""
    theta *=np.pi/180.
    v = prev/np.linalg.norm(prev)
    # find "mostly orthogonal" vector to prev
    a = np.zeros((3,))
    a[np.argmin(np.abs(prev))]=1
    # find orthonormal coordinate system {x_hat, y_hat, v}
    x_hat = np.cross(a,v)/np.linalg.norm(np.cross(a,v))
    y_hat = np.cross(v,x_hat)
    # draw random rotation 
    phi = np.random.uniform(0.,2.*np.pi) 
    # determine vector (random rotation + rotation with theta to guarantee the right angle between v,w)
    w = np.sin(theta)*np.cos(phi)*x_hat + np.sin(theta)*np.sin(phi)*y_hat + np.cos(theta)*v
    w *=R
    return w

def random_coords(N,model,theta):
    coords = np.zeros((int(N),3))
    coords[1]=[1,0,0]
    for i in range(2,int(N)):
        if model == 'freely_jointed':
            n = random_point_on_shpere(1.0)
            new = coords[i-1]+n
        if model == 'freely_rotating':
            prev = coords[i-2]-coords[i-1]
            n = random_point_on_cone(1.0,theta,prev)
            new = coords[i-1]+n
        coords[i]=new
    return coords


def random_coords_2D(N,type):
    coords = np.zeros((int(N),2))
    coords[1]=[1.,0.]
    for i in range(2,int(N)):
        if type=='continous':
            n = random_point_on_circle(1.0)
        elif type=='lattice':
            n = random_point_on_lattice(1.0)
        new = coords[i-1]+n
        coords[i]=new
    return coords

def random_walk(N):
    steps = np.random.randint(low = -1, high = 1, size = (N))*2 +1 
    random_trajectory = np.cumsum(steps)
    random_trajectory = np.concatenate([[0],random_trajectory])
    
    return random_trajectory

def calculate_overlaps(coords):
    d = cdist(coords,coords)
    i = np.arange(len(coords)).reshape(len(coords),1)
    u = cdist(i,i)
    q = np.where(np.logical_and(u > 1,d<1) , True, False)
    l = np.argwhere(q)
    l = np.unique(np.vstack((l[:,0],l[:,1])))
    return coords[l.astype(int)]
    


n = ipw.widgets.IntText(
    value=10,
    description='N:',
    disabled=False,
continuous_update=False)

m = ipw.widgets.RadioButtons(
    options=['lattice', 'continous'],
    description='type:',
    disabled=False
)  

%matplotlib notebook
def random2D(n,type,draw_beads):
    a = random_coords_2D(n,type)
    plt.figure()
    plt.plot(a[:,0],a[:,1],color='black',linewidth=0.2)
    if draw_beads:
        plt.scatter(a[:,0],a[:,1],color='red')
    plt.show()
    return a

def random3D(n,draw_beads):
    a = random_coords(n,'freely_jointed',0)
    ipv.figure(height=400,width=600)
    ipv.pylab.plot(a[:,0],a[:,1],a[:,2],color='black')
    if draw_beads:
        ipv.pylab.scatter(a[:,0],a[:,1],a[:,2],color='red')
    ipv.pylab.xyzlabel(' ',' ',' ')
    
    max = np.max(a)
    min = np.min(a)
    ipv.pylab.xlim(min, max)
    ipv.pylab.ylim(min, max)
    ipv.pylab.zlim(min, max)
    ipv.show()
    return a

ToggleButton(value=False, description='Show code')

# Random walks 
## 1D

In [2]:
# 1D random walk

n = ipw.widgets.IntSlider(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='# walks:',
    disabled=False)

N = ipw.widgets.IntSlider(
    min=1,
    max=1000,
    step=1,
    value=10,
    description='Length N:',
    disabled=False)

def g(N=100,n=1):
    plt.figure()
    for i in np.arange(n):
        a = random_walk(N)
        plt.step(np.arange(N+1),a)
       # plt.scatter(np.arange(N+1),a)
    plt.xlabel('N')
    plt.ylabel('x')
    plt.show()

q = ipw.interactive(g,{'manual': True}, n=n,N=N );
display(q)

#q = ipw.interactive(g,{'manual': True}, n=n2,N=N2,draw_approx=appr);
#display(q)

interactive(children=(IntSlider(value=10, description='Length N:', max=1000, min=1), IntSlider(value=1, descri…

In [3]:
n2 = ipw.widgets.IntSlider(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='# walks:',
    disabled=False)

N2 = ipw.widgets.IntSlider(
    min=1,
    max=1000,
    step=1,
    value=10,
    description='Length N:',
    disabled=False)

appr = ipw.widgets.Checkbox(
    value=False,
    description='Draw Gaussian approximation',
    disabled=False,
    indent=False
)

def g2(N=100,n=1,draw_approx=False):
    f, (ax1, ax2) = plt.subplots(1, 2,sharey=True)
    end_points=[]
    for i in np.arange(n):
        a = random_walk(N)
        end_points.append(a[-1])
        ax1.step(np.arange(N+1),a,alpha=0.1,c='b')
    ax1.set_xlabel('N')
    ax1.set_ylabel('x')
    ax2.hist(end_points,orientation='horizontal',density=True)
    ax2.set_xlabel('norm. counts')
    ax2.set_ylabel('x_end')
    if draw_approx==True:
        x =np.linspace(np.min(end_points),np.max(end_points))
        y = 1./np.sqrt(2*np.pi*N)*np.exp(-x**2/(float(2*N)))
        ax2.plot(y,x)
    plt.tight_layout()
    plt.show()
    
q = ipw.interactive(g2,{'manual': True}, n=n2,N=N2,draw_approx=appr);
display(q)    

interactive(children=(IntSlider(value=10, description='Length N:', max=1000, min=1), IntSlider(value=1, descri…

## 2D

In [4]:
walk = ipw.interactive(random2D,{'manual': True},type=m,n=n,draw_beads=True);
display(walk)



interactive(children=(IntSlider(value=1, description='# walks:', max=1000, min=1), RadioButtons(description='t…

## 3D

In [5]:
walk = ipw.interactive(random3D,{'manual': True},n=n,draw_beads=True);
display(walk)

interactive(children=(IntSlider(value=1, description='# walks:', max=1000, min=1), Checkbox(value=True, descri…

In [6]:
def g4(N=100,n=1,draw_gaussian=False):
    f, (ax1, ax2) = plt.subplots(1, 2)
    ipv.figure(height=400,width=600)
    end_lenghts=[]
    rainbow  = matplotlib.cm.get_cmap('rainbow')
    colors = rainbow(np.linspace(0.0,1.0,n))
    max = -1000
    min = 1000
    for i in np.arange(n):
        a = random_coords(N,'freely_jointed',0.0)
        max = np.max([max,np.max(a)])
        min = np.min([min,np.min(a)])
        end_to_end_dist_length = np.linalg.norm(a[-1]-a[0])
        end_lenghts.append(end_to_end_dist_length)
    
        ipv.pylab.plot(a[:,0],a[:,1],a[:,2],color=colors[i],alpha=0.1)
  
    ipv.pylab.xyzlabel(' ',' ',' ')

    ipv.pylab.xlim(min, max)
    ipv.pylab.ylim(min, max)
    ipv.pylab.zlim(min, max)
    
    ax1.scatter(np.arange(n),end_lenghts,color=colors)
    
    ax1.set_xlabel('n')
    ax1.set_ylabel('|R|')
    
    hist, bin_edges = np.histogram(end_lenghts)
    factor = 4./3. * np.pi 
    radii = bin_edges[:-1]
    radii_right = bin_edges[1:]
    hist_normalized = hist* (radii_right**2 - radii**2)
    
    center = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax2.bar(center,hist_normalized)

    ax2.set_xlabel('norm. counts')
    ax2.set_ylabel('|R|')
    if draw_gaussian==True:
        x =np.linspace(0,np.max(end_lenghts))
        y = (3./(2*np.pi*N))**(3./2.)*np.exp(-3*x**2/(float(2*N)))*x**2
        ax2.plot(x,y*np.max(hist_normalized)/np.max(y))
    plt.tight_layout()
    ipv.show()
    plt.show()
    
# radnom walk statistics in 3D    
q = ipw.interactive(g4,{'manual': True}, n=(1,1000,1),N=(1,1000,1),draw_gaussian=False);
display(q)    

interactive(children=(IntSlider(value=100, description='N', max=1000, min=1), IntSlider(value=1, description='…